### OLS 선형회귀 

OLS 선형회귀 포커스 프로젝트에 온것을 환영합니다^^ <br>
여기에서는 OLS 선형회귀를 적용하여 자동차의 연비에 대한 물음에 답하게 됩니다. <br>

#### 목표
- 선형대수학을 응용한 선형회귀 계산.
- 회귀계수, 회귀계수의 표준오차, 통계량, t-테스트.
- 정보량, 결정계수 계산.
- 모형의 최적화와 데이터에 대한 물음에 답하기.

선형회귀문제 연비구하기....
1. 수동/자동 중에 연비가 뭐가 더 좋을까? 
>> 이 분석에서 타 스펙들은 동등한 조건에서 자동, 수동을 비교하기 위해 사용됨
>> 왜?! 다른 스펙이 같아야 제대로 된 비교를 하니까.

그럼, 일단 수동/자동만 독립변수로 채택해서 모델을 만들고, 테스트를 한다면,
수동 변속기의 연비가 높게 나온다. 과연 이게 맞을까?
나머지 스펙들은 고려하지 않았기 때문에 틀리다.

그렇다면 다른 스펙들을 계속해서 독립변수로 추가해서 모델을 만든다면 어떤 결과가 나타날까?

프로그래밍-
잔차 R^2 표준오차 요약 AIC 를 객체 안에 속성으로..




#### 필요한 패키지를 읽어 옵니다

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import t
import os
%matplotlib inline

#### 트레이닝/테스팅 데이터 읽어오기

In [ ]:
os.chdir(r"D:\python_ML\python_ml_basic\data")
df = pd.read_csv('data_mtcars.csv', header='infer',encoding='latin1')

#### Linear Regression 클래스 정의

In [ ]:
class linearRegression:
    def __init__(self, input_df):
        self.df = input_df
        self.all_features = list(df.drop(columns=['model','mpg']).columns)
        return
        
    def train(self, features):
        input_X = ((self.df).drop(columns=['model','mpg'])).loc[:,features]
        input_Y = df.loc[:,'mpg']
        ones_column = np.ones((input_X.shape[0],1))
        X = np.concatenate((ones_column,input_X),axis=1)
        Y = np.array(input_Y).reshape(-1,1)
        self.N = X.shape[0]
        self.p = X.shape[1]
        self.df = self.N - self.p
        
        # Regression Coefficients
        self.beta = np.dot(np.linalg.inv(np.dot(X.T,X)),np.dot(X.T,Y))
        
        # Residuals
        self.residuals = Y - np.dot(X,self.beta)
        
        # R^2
        SSE = np.dot((self.residuals).T, self.residuals)
        SST = np.dot((Y - Y.mean()).T, (Y-Y.mean()))
        self.rsquared = 1.0 - SSE[0][0]/SST[0][0]
        
        # Standard Errors
        s2 = SSE/self.df
        Qinv = np.linalg.inv(np.dot(X.T,X))
        self.se = np.sqrt(s2*np.diag(Qinv).reshape(-1,1))
        self.tvals = self.beta/self.se
        self.pvals = (1.0-t.cdf(np.abs(self.tvals), self.df))*2.0
        
        # Summary
        self.summary=pd.DataFrame(np.concatenate([self.beta,self.se,self.tvals,self.pvals],axis=1))
        self.summary.columns =['Beta','Std. Error','t-value','p-value']
        self.summary.index = ['intercept'] + features 
        
        # AIC
        self.AIC = (1.0 + np.log(2*np.pi) + np.log(SSE[0][0]/self.N)) + 2.0*self.p/self.N
        return  
    
    def output(self):
        print(self.summary)
        print("--------------------------------------------------------------")
        print("DF  : %d" % self.df)
        print("R^2 : %f" % self.rsquared)
        print("AIC : %f" % self.AIC)


#### 상관계수 행렬

In [ ]:
np.round(df.corr(),3)

#### 선형회귀 예

In [ ]:
LR = linearRegression(df)
LR.train(['am'])
LR.output()

#### 이제는 모형을 최적화하여 변속기 타입에 의한 연비차이에 대해서 답하시오. 